In [ ]:
# !pip install --upgrade google-genai arxiv beautifulsoup4 requests readability-lxml

In [ ]:
from getpass import getpass
import os

api_key= getpass("Paste your Google Gemini APi Key here:")
os.environ["GEMINI_API_KEY"]=api_key

Paste your Google Gemini APi Key here:··········


In [ ]:
from google import genai

client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [ ]:
resp = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Summarize in one sentence. Why is AI important?"
)
print(resp.text)


AI is important because it automates complex tasks, analyzes vast data for insights, and drives innovation across virtually all sectors, fundamentally reshaping how we live and work.


In [ ]:
import arxiv
def search_arxiv(query, max_results=5):
  search= arxiv.Search(
      query=query,
      max_results=max_results,
      sort_by=arxiv.SortCriterion.Relevance
  )

  results=[]
  for result in search.results():
    results.append({
        "title": result.title,
        "summary": result.summary,
        "authors": [a.name for a in result.authors],
        "pdf_url" : result.pdf_url,
        "id" : result.get_short_id()
    })

  return results

  papers=search_arxiv("Multimodal transformers", max_results=3)
  for p in papers:
    print(p['title'])

In [ ]:
import requests
from bs4 import BeautifulSoup

def fetch_page_text(url, max_chars=4000):
  try:
    r = requests.get(url, timeout=10, headers={"User-Agent": "research-agent/1.0" })
    r.raise_for_status()
  except Exception as e:
    return ""
  soup = BeautifulSoup(r.text, "html.parser")
  paragraphs= [p.get_text(seperator= "  ", strip=True) for p in soup.find_all("p")]
  content = "\n\n".join(paragraphs)
  return content[:max_chars]

In [ ]:
def chunk_text(text, max_chars=3000):
  chunks=[]
  start=0
  while start < len(text):
    end = min(len(text),start + max_chars)
    chunks.append(text[start:end])
    start = end
  return chunks

In [ ]:
SYSTEM_PROMPT=(
    "You are a concise research assitant. For each text provided, return:"
    "1) a one paragraph summary (3-5) sentences, 2) three bullet key contribution/findings,"
    "and 3) a suggested short title. Be factual  and include no hallucinated facts."
)

def summarize_chunk(chunk_text):
  prompt=SYSTEM_PROMPT + "\n\n Text to summarize: \n" + chunk_text
  resp = client.models.generate_content(
      model="gemini-2.5-flash",
      contents=prompt
  )
  return resp.text

In [ ]:
def summarize_text_long(text):
  chunks = chunk_text(text, max_chars=3000)
  summaries=[]
  for c in chunks:
    s=summarize_chunk(c)
    summaries.append(s)
  if len(summaries) ==1:
    return summaries[0]
  else:
    combined ="\n\n".join(summaries)
    final_prompt = SYSTEM_PROMPT + "\n\n Combine the following chunk summaries into a single concise summary: \n\n" + combined
    final_resp= client.models.generate_content(model="gemini-2.5-flash", contents=final_prompt)
    return final_resp.text

In [ ]:
query = "Ai and impact on Privacy"
papers = search_arxiv(query, max_results=5)

for p in papers:
  print("\n---")
  print("Title", p['title'])
  text_to_summarize= p['summary']
  summary = summarize_text_long(text_to_summarize)

  print("Summary: \n", summary)
  print("PDF: ",p['pdf_url'])

/tmp/ipython-input-280155989.py:10: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():



---
Title The Global Impact of AI-Artificial Intelligence: Recent Advances and Future Directions, A Review
Summary: 
 1) This article synthesizes recent research on the global impact of Artificial Intelligence (AI), exploring its potential benefits and risks across various societal sectors. It details AI's implications for the economy, ethics, society, security, privacy, and job displacement. The text specifically highlights ethical concerns such as bias, security vulnerabilities, and privacy violations inherent in AI development. To ensure the responsible development and deployment of AI, the article stresses the importance of collaboration among government, industry, and academia, alongside public engagement and education.

2) Key contributions/findings:
*   AI is an emerging technology with the potential to transform society across economic, healthcare, and transportation sectors, bringing both benefits and significant risks.
*   Key challenges in AI development include ethical con